In [1]:
import requests
import finnhub
import warnings
import time
import numpy as np
import pandas as pd
from yahoo_fin import stock_info
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import date,timedelta
from yahoo_fin import stock_info

In [2]:
warnings.filterwarnings('ignore')

# NASDAQ 100 List

In [3]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'

In [4]:
html = requests.get(url)

In [5]:
soup = BeautifulSoup(html.content)

In [6]:
table = soup.find_all('table', id = 'constituents')

In [7]:
nasdaq_list = pd.read_html(str(table[0]))[0]

In [8]:
nasdaq_list.head()

,Company,Ticker,GICS Sector,GICS Sub-Industry
0,Activision Blizzard,ATVI,Communication Services,Interactive Home Entertainment
1,Adobe,ADBE,Information Technology,Application Software
2,ADP,ADP,Information Technology,Data Processing & Outsourced Services
3,Airbnb,ABNB,Consumer Discretionary,Internet & Direct Marketing Retail
4,Align,ALGN,Health Care,Health Care Supplies


# S&P 100 list

In [45]:
url = 'https://en.wikipedia.org/wiki/S%26P_100'

html = requests.get(url)

soup = BeautifulSoup(html.content)

table = soup.find_all('table', id = 'constituents')

sp_list = pd.read_html(str(table[0]))[0]

In [41]:
#sp_list = sp_list.Symbol.values
#nasdaq_list = nasdaq_list.Ticker.values

In [47]:
sp = [ticker for ticker in sp_list.Symbol.values if ticker not in nasdaq_list]

In [48]:
sp_list

,Symbol,Name,Sector
0,AAPL,Apple,Information Technology
1,ABBV,AbbVie,Health Care
2,ABT,Abbott,Health Care
3,ACN,Accenture,Information Technology
4,ADBE,Adobe,Information Technology
...,...,...,...
96,VZ,Verizon,Communication Services
97,WBA,Walgreens Boots Alliance,Consumer Staples
98,WFC,Wells Fargo,Financials
99,WMT,Walmart,Consumer Staples


# News scraping

In [21]:
finnhub_client = finnhub.Client(api_key = "c7i4q2qad3if83qgdfl0")

In [22]:
def get_company_news(ticker):
    news = []
    initial_date = datetime.now().date()
    for i in range(60):
        offset = (pd.to_datetime(initial_date) - pd.DateOffset(days = 5)).strftime('%Y-%m-%d')
        news_iter = finnhub_client.company_news(ticker, _from = offset, to = initial_date)
        news += news_iter

        initial_date = (pd.to_datetime(offset) - pd.DateOffset(days = 1)).strftime('%Y-%m-%d')

    return news

In [23]:
#for illustrative purposes only 2 tickers are scraped
data = {}
tickers = sp_list
for ticker in tickers:
    start = time.time()
    data[ticker] = get_company_news(ticker)
    end = time.time() - start
    print(f'{ticker} news is parsed!')
    #print(f'Sleeping for {np.round((62 - end), 0)} seconds...')
    #time.sleep((60 - end))
    if ticker != tickers[-1]:
        time.sleep(70)
    else:
        continue

ABBV news is parsed!
ABT news is parsed!
ACN news is parsed!
AIG news is parsed!
AMT news is parsed!
AXP news is parsed!
BA news is parsed!
BAC news is parsed!
BK news is parsed!
BLK news is parsed!
BMY news is parsed!
BRK.B news is parsed!
C news is parsed!
CAT news is parsed!
CL news is parsed!
COF news is parsed!
COP news is parsed!
CRM news is parsed!
CVS news is parsed!
CVX news is parsed!
DD news is parsed!
DHR news is parsed!
DIS news is parsed!
DOW news is parsed!
DUK news is parsed!
EMR news is parsed!
F news is parsed!
FDX news is parsed!
GD news is parsed!
GE news is parsed!
GM news is parsed!
GS news is parsed!
HD news is parsed!
IBM news is parsed!
JNJ news is parsed!
JPM news is parsed!
KO news is parsed!
LIN news is parsed!
LLY news is parsed!
LMT news is parsed!
LOW news is parsed!
MA news is parsed!
MCD news is parsed!
MDT news is parsed!
MET news is parsed!
MMM news is parsed!
MO news is parsed!
MRK news is parsed!
MS news is parsed!
NEE news is parsed!
NKE news is pa

In [24]:
#checking the correctness
data.keys()

dict_keys(['ABBV', 'ABT', 'ACN', 'AIG', 'AMT', 'AXP', 'BA', 'BAC', 'BK', 'BLK', 'BMY', 'BRK.B', 'C', 'CAT', 'CL', 'COF', 'COP', 'CRM', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'F', 'FDX', 'GD', 'GE', 'GM', 'GS', 'HD', 'IBM', 'JNJ', 'JPM', 'KO', 'LIN', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDT', 'MET', 'MMM', 'MO', 'MRK', 'MS', 'NEE', 'NKE', 'ORCL', 'PFE', 'PG', 'PM', 'RTX', 'SO', 'SPG', 'T', 'TGT', 'TMO', 'UNH', 'UNP', 'UPS', 'USB', 'V', 'VZ', 'WFC', 'WMT', 'XOM'])

### Function for news dataset

In [49]:
# в 19 строчке надо определить время, после которого отсекать
def get_news_dataset(ticker):
    #extracting news from dictionary 
    news = pd.DataFrame(data[ticker])
    news['datetime'] = news['datetime'].apply(lambda date: datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S'))

    #only news with ticker occurance
    company_name = sp_list[sp_list['Symbol'] == ticker]['Name'].to_list()[0]
    news = news[news.headline.str.contains(company_name)].reset_index(drop = True)

    #GMT-to-EST correction
    news['datetime'] = news['datetime'].apply(lambda date: pd.to_datetime(date) - pd.DateOffset(hours = 5))
    
    #next day correction
    timestamps = news['datetime'] 
    timestamps_new = []
    for date in timestamps:
        date = date.strftime('%Y-%m-%d %H:%M:%S')
        if pd.to_datetime(date) > pd.to_datetime(f'{date[:10]} 10:00:00'):
            date = pd.to_datetime(date)
            date_new = (date - timedelta(hours = date.hour, minutes = date.minute,
                    seconds = date.second)) + timedelta(days = 1)
            timestamps_new.append(date_new.strftime('%Y-%m-%d %H:%M:%S'))
        else:
            timestamps_new.append(date)


    news['datetime'] = timestamps_new
    
    news = news[['datetime', 'headline', 'source', 'summary']]
    news['datetime'] = news['datetime'].apply(lambda date: pd.to_datetime(date).strftime('%Y-%m-%d'))
    
    #trading days only
    bdays = pd.bdate_range(start = news['datetime'].to_list()[-1], 
               end = news['datetime'][0])

    bdays = pd.DataFrame(bdays, columns = ['datetime'])

    bdays.datetime = bdays.datetime.astype('str')

    news_filtered = pd.merge(news, bdays, how = 'right')
    
    #dropping dublicates
    news_filtered.drop_duplicates(['headline'], ignore_index = True, inplace = True)
    
    return news_filtered
    
    


# Financials

#### Relative Strength Index

In [26]:
#здесь выбрать какое среднее брать - обычное или эксп
def rsi(historical_data, window):
    changes = historical_data['adjclose'].diff(periods = 1)
    gain, loss = changes.clip(lower = 0), changes.clip(upper = 0).abs()
    
    #rsi based on SMA
    rolling_gain = gain.rolling(window, closed = 'left').mean()
    rolling_loss = loss.rolling(window, closed = 'left').mean()
    
    rs = rolling_gain / rolling_loss
    rsi_sma = 100.0 - (100.0 / (1.0 + rs))
    
    #rsi based on EMA
    rolling_gain_exp = rolling_gain.ewm(span = window).mean()
    rolling_loss_exp = rolling_loss.ewm(span = window).mean()

    rs_exp = rolling_gain_exp / rolling_loss_exp
    rsi_ema = 100.0 - (100.0 / (1.0 + rs_exp))
    
    return rsi_sma, rsi_ema

#### Financial indicators for each day

In [27]:
def financial_indicators(historical_data):
    df = {}
    data = historical_data['adjclose']
    df['datetime'] = historical_data['adjclose'].index
    df['adjclose'] = data
    #10 days Moving Average
    df['ma_10'] = data.rolling(window = 10, closed = 'left').mean()[10:]
    #20 days Moving Average
    df['ma_20'] = data.rolling(window = 20, closed = 'left').mean()[20:]
    #10 days Moving Average
    df['ma_30'] = data.rolling(window = 30, closed = 'left').mean()[30:]
    
    #12 days Exponential Moving Average
    df['ema_12'] = data.ewm(span = 12).mean()[1:]
    df['ema_12'] =  df['ema_12'].shift()
    #26 days Exponential Moving Average
    df['ema_26'] = data.ewm(span = 26).mean()[1:]
    df['ema_26'] =  df['ema_26'].shift()
    #6 days Relative Strength Index
    df['rsi_6_sma'], df['rsi_6_ema'] = rsi(historical_data, window = 6)
    #12 days Relative Strength Index
    df['rsi_12_sma'], df['rsi_12_ema'] = rsi(historical_data, window = 12)
    #24 days Relative Strength Index
    df['rsi_24_sma'], df['rsi_24_ema'] = rsi(historical_data, window = 24)
    
    final_df = pd.DataFrame(df)
   
    return final_df

In [28]:
def get_financials_dataset(ticker):
    #historical data
    historical_data = stock_info.get_data(ticker,
                                      start_date = pd.to_datetime(news_filtered['datetime'][0]) - 
                                      pd.DateOffset(days = 100), 
                                      end_date =  news_filtered['datetime'].to_list()[-1],
                                      index_as_date = True, 
                                      interval = '1d')
    
    #financial indicators
    financials = financial_indicators(historical_data)
    financials = financials[financials['datetime'] >= pd.to_datetime(news_filtered.datetime[0])]
    financials['datetime'] = financials['datetime'].apply(lambda date: date.strftime('%Y-%m-%d'))
    
    #reseting the index for further concatenation with news dataset
    financials.index = financials.index.astype('object')
    fin_index = pd.Series(financials.index)
    fin_index = fin_index.apply(lambda fin : fin.strftime('%Y-%m-%d'))
    financials.set_index(fin_index, drop = True, inplace = True)
    
    return financials

### Reshaping the data to long format

In [29]:
def reshape_news_data(news_filtered):
    news_filtered['RANK'] = news_filtered.groupby("datetime")["datetime"].rank(method="first", ascending=True)
    news_filtered['RANK'] = news_filtered['RANK'].astype('int')
    news_reshaped = news_filtered.pivot(index = 'datetime', values = 'headline', columns = 'RANK')
    news_reshaped.columns = [f'news_{index}' for index in news_reshaped.columns]
    
    return news_reshaped

# Getting Target

In [30]:
def get_target(historical_data, predict_trend = True):
    historical_data['diff'] = historical_data['adjclose'] - historical_data['open']
    movements = historical_data['diff'].apply(lambda price: 1 if price > 0 else 0)
    target = pd.DataFrame({'date':historical_data['date'], 'target' : movements}).set_index('date')
    
    target.index = target.index.astype('object')

    target_index = pd.Series(target.index)
    target_index = target_index.apply(lambda fin : fin.strftime('%Y-%m-%d'))
    target.set_index(target_index, drop = True, inplace = True)
    if predict_trend:
        #trend
        trend = []
        i = 0
        i_initial = 0
        for i in range(len(target) - 1):
            try:
                if target.target[i]:
                    while target.target[i] == 1:
                        i += 1

                else:
                    while target.target[i] == 0:
                        i += 1

                trend.append(i - i_initial)

                i_initial += 1
                i = 0
            except:
                break
        #appending trend
        target['trend'] = np.NaN
        index = len(target) - len(trend)
        target['trend'][:-index] = trend
        target = target.iloc[:-index, :]

        target.trend = target.trend.astype('int')

        return target
    else:
        return target

# Final concatenation

In [52]:
tickers_dict = {}
error_list = []
for ticker in data.keys():
    try:
        #news 
        news_filtered = get_news_dataset(ticker)

        #historical prices
        historical_data = stock_info.get_data(ticker,
                                          start_date = pd.to_datetime(news_filtered['datetime'][0]) - 
                                          pd.DateOffset(days = 100), 
                                          end_date =  news_filtered['datetime'].to_list()[-1],
                                          index_as_date = False, 
                                          interval = '1d')

        #financials
        financials = get_financials_dataset(ticker)

        #news reshaped
        news_reshaped = reshape_news_data(news_filtered)

        #concatenate news and financials
        final_dataframe = pd.concat([news_reshaped, financials], axis = 1, join = 'inner')
        final_dataframe.drop(['datetime', 'datetime'], axis = 1, inplace = True)

        #target
        target = get_target(historical_data, predict_trend = True)
        target['target_3_days_previous'] = target.target.rolling(3, closed = 'left').mean()
        target['target_5_days_previous'] = target.target.rolling(5, closed = 'left').mean()
        #concatenate news + financials and target
        final_dataframe = pd.concat([final_dataframe, target], axis = 1, join = 'inner')

        #ticker indicator
        final_dataframe['ticker'] = ticker

        #appending to dictionary for further concatenation
        tickers_dict[ticker] = final_dataframe
    except:
        print(f"{ticker} returns error!")
        error_list.append(ticker)
        
        
combined_data = pd.concat([tickers_dict[key] for key in tickers_dict.keys()], axis = 0)

BRK.B returns error!


# S&P return

In [53]:
market_data = stock_info.get_data('^GSPC',
                                          start_date = pd.to_datetime(news_filtered['datetime'][0]) - 
                                          pd.DateOffset(days = 100), 
                                          end_date =  news_filtered['datetime'].to_list()[-1],
                                          index_as_date = False, 
                                          interval = '1d')
historical_data = market_data
historical_data['diff'] = historical_data['adjclose'] - historical_data['open']
movements = historical_data['diff'].apply(lambda price: 1 if price > 0 else 0)
target = pd.DataFrame({'date':historical_data['date'], 'market_return' : movements}).set_index('date')

target.index = target.index.astype('object')

target_index = pd.Series(target.index)
target_index = target_index.apply(lambda fin : fin.strftime('%Y-%m-%d'))
target.set_index(target_index, drop = True, inplace = True)

target['market_3_days_previous'] = target.market_return.rolling(3, closed = 'left').mean()
target['market_5_days_previous'] = target.market_return.rolling(5, closed = 'left').mean()

combined_data = combined_data.join(target)

In [54]:
combined_data.drop(columns = ['rsi_12_ema', 'rsi_24_ema'], axis = 1, inplace = True)

In [55]:
combined_data.columns

Index(['news_1', 'news_2', 'news_3', 'news_4', 'news_5', 'news_6', 'news_7',
       'news_8', 'news_9', 'news_10',
       ...
       'news_81', 'news_82', 'news_83', 'news_84', 'news_85', 'news_86',
       'news_87', 'market_return', 'market_3_days_previous',
       'market_5_days_previous'],
      dtype='object', length=105)

In [56]:
combined_data.head(5)

,news_1,news_2,news_3,news_4,news_5,news_6,news_7,news_8,news_9,news_10,...,news_81,news_82,news_83,news_84,news_85,news_86,news_87,market_return,market_3_days_previous,market_5_days_previous
2021-03-09,A GE-AerCap Deal Would Add Pressure on Boeing ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.333333,0.2
2021-03-09,Bank of America's Dividend Lags Its Peers -- b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.333333,0.2
2021-03-09,"Don’t chase Caterpillar and Deere, says trader...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.333333,0.2
2021-03-09,Colgate-Palmolive Honored as one of Fast Compa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.333333,0.2
2021-03-09,Dow up 175 points on gains in Salesforce.com I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.333333,0.2
